In [1]:
import numpy as np
import random 
from random import randint
import time
import pygame
import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras.layers import Dense, Input, Flatten


board_size = 7
sizeX = 770 / board_size
sizeY = 770 / board_size
turns = {0:[-1, 0], 1:[0, 1], 2:[1, 0], 3:[0, -1]}


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


Using TensorFlow backend.


In [2]:
def init_model():    
    inp = Input(shape=(49, 1))
    flat = Flatten()(inp)
    x1 = Dense(7, activation="relu")(flat) 
    out = Dense(4, activation="softmax")(x1) 
    model = Model(input=inp, output=out)
    return model

In [3]:

def predict(vision, model): 
    """ Prediction move using board """
    vision = np.reshape(vision, (1, 49, 1))  
    prediction = model.predict(vision) 
    return prediction

In [4]:
def train_play(  weights=0):
    """ 1 epoch of snake life """
    alife = True
    turn = 0 
    model = init_model()
    if not weights == 0:
        model.set_weights(weights)
    player = snake()
    player.set_board()
    while(alife == True and turn < 150):
        turn += 1
        board = player.set_board()
        prediction = predict( board, model)
        pick = turns[np.argmax(prediction)]  
        alife = player.move(pick)
    score = player.get_score()
    return score , model 

In [15]:
def new_weights(w, score):
    """ generating new random weights values""" 
    for i, _ in enumerate(w): 
        if np.shape(w) == (7,):
            # w[i] = w[i] = w[i] + random.uniform(-1, 1) * (1 / np.exp(score))
            w[i] = w[i] = w[i] + random.uniform(-1, 1) * (1 / np.exp(score))
        else:
            for j, _ in enumerate(w[i]): 
                w[i][j] = w[i][j] + random.uniform(-1, 1)  
                w[i][j] = w[i][j] + random.uniform(-1, 1)  
    return np.array(w)

In [8]:
def training_loop(epoches=10, delay=1):
   
    best_model = None
    best_score_tmp = 0.1
    best_score = 0
    for i in range(epoches):
     
        for j in range(7):
            if not i == 0: 
                w_new = new_weights(w, best_score) 
                #w_new[1] = list( map(lambda x: x + random.uniform(-1, 1) * 0.7 / best_score, w[1]) ) 
            else:
                 w_new = 0
                
            score, model = train_play( weights=w_new)
            print(j, " score = ", score)
            if score > best_score:
                best_model = model
                best_score_tmp = score 
                best_score = score
        print("best score = ", best_score_tmp)
        
        visual(best_model, delay=delay)
        w = best_model.get_weights()
        w = np.array(list(w))  
        best_score_tmp = 0.1
        
   # print(w)

   # best_model.set_weights(w)
    
    visual(best_model, delay=delay)
    
    return best_model

In [9]:
def draw(win, board):  
    """drawing board to screen"""
    for i in range(0, np.shape(board)[0]): 
        for j in range(0, np.shape(board)[1]):    
            if board[i][j]== 0 :
                pygame.draw.rect(win,(30, 30, 30),(i*sizeX, j*sizeY, sizeX - 4, sizeY - 4)) 
            if board[i][j]== 0.5 :
                pygame.draw.rect(win,(0, 220, 0),(i*sizeX, j*sizeY, sizeX - 4, sizeY- 4)) 
            if board[i][j]== 0.7 :
                pygame.draw.rect(win,(0, 160, 0),(i*sizeX, j*sizeY, sizeX - 4, sizeY- 4))
            if board[i][j]== 1 :
                pygame.draw.rect(win,(255, 0, 0),(i*sizeX, j*sizeY, sizeX - 4, sizeY- 4)) 


In [10]:
class snake():
    """ main game class """
    def __init__(self):
        self.segments = np.array([[2, 2],[3, 2],[4, 2]])
        self.head = np.array([5, 2])
        self.apple = np.array([1, 1])
        self.score = 0

    def set_board(self):
        """Setting game objects to the board"""
        self.board = np.zeros((board_size, board_size))
        self.board[self.head[0]][self.head[1]] = 0.7
        self.board[self.apple[0]][self.apple[1]] = 1
        for i in self.segments:
            self.board[i[0]][i[1]] = 0.5
        return self.board    

    def move(self, turn):
        # for i in seegments[-1]
        new_head_pos = self.head + turn  

        if (new_head_pos == self.apple).all():
            self.take_apple()
        elif ((new_head_pos>board_size-1).any() or (new_head_pos<0).any() or (self.board[new_head_pos[0]][new_head_pos[1]] == 0.5)):
           # print("end")
            return False
        else:   # segment moving
            for i in range(len(self.segments)-1):
                self.segments[i] = self.segments[i+1]  
            else:
                self.segments[i+1] = self.head.copy()    
            self.head = new_head_pos
        self.score += 0.2
        self.set_board()
        return True
    def take_apple(self):
        """ Generating new segment of snake and new apple in random place"""
        self.new_segment()
        self.head = self.apple.copy()
        while( self.board[self.apple[0]][self.apple[1]] != 0):
            self.apple = np.array([randint(0, board_size-1), randint(0, board_size-1)])
        self.score +=5

    def get_score(self):
        return self.score

    def new_segment(self):
        """ Addition a new sigment to the snake"""
        self.segments = np.append(self.segments, [self.head.copy()] , axis = 0)
        self.head = self.apple.copy() 

In [11]:

def visual(model, delay=0.5, max_turns=150):
    """to see what snake ~doing"""
    player = snake()
    board = player.set_board() 


    pygame.init() 
    win = pygame.display.set_mode((770,770))  
    alife = True
    turn = 0 
    player = snake()
    player.set_board()
    while(alife == True and turn < max_turns):
        turn += 1
        board = player.set_board()
        prediction = predict(board, model)
        pick = turns[np.argmax(prediction)] 
        draw(win, board)

        pygame.display.update()

        alife = player.move(pick)
        time.sleep(delay) 
    score = player.get_score()
    print(score)
    pygame.display.quit()
    pygame.quit()

In [16]:
random.seed(2) 
ddd = training_loop(25, delay=0.3)
 


D:\miniconda\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


0  score =  0.4
1  score =  0
2  score =  0
3  score =  0.2
4  score =  0.4
5  score =  0.4
6  score =  0.2
best score =  0.4
0.4


D:\miniconda\lib\site-packages\ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
D:\miniconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


0  score =  0.4
1  score =  6.2
2  score =  0
3  score =  0
4  score =  0
5  score =  0
6  score =  0
best score =  6.2
6.2
0  score =  1.4
1  score =  0
2  score =  0.4
3  score =  0.4
4  score =  0.4
5  score =  0.4
6  score =  0.4
best score =  0.1
6.2
0  score =  1.4
1  score =  0.4
2  score =  0.4
3  score =  0.4
4  score =  0.4
5  score =  0.4
6  score =  0.4
best score =  0.1
6.2
0  score =  0.6000000000000001
1  score =  0
2  score =  0.4
3  score =  0.4
4  score =  0.4
5  score =  0.4
6  score =  0.4
best score =  0.1
6.2
0  score =  0.4
1  score =  0.4
2  score =  0.2
3  score =  0.4
4  score =  0.2
5  score =  0.2
6  score =  0.2
best score =  0.1
6.2
0  score =  6.2
1  score =  0.4
2  score =  0.4
3  score =  0.4
4  score =  0.4
5  score =  0.4
6  score =  0.4
best score =  0.1
6.2


KeyboardInterrupt: 

In [ ]:
a = [1,2,3]
b  = a.copy()
b[2] = 5
print(a)

In [ ]:
x = ["0", "1", "2"]
y = ''.join(x) # converting list into string
z = int(y)
print(z)